<a href="https://colab.research.google.com/github/BrainConnection/Small_Project/blob/main/Quantum%20Squeezing%20Induced%20Optical%20Nonreciprocity%20Reproduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Environment Setting



In [1]:
!pip install qutip
!pip install numpy
!pip install matplotlib
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 14.0 MB/s eta 0:00:00


In [2]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate

# Hamiltonian (FW Case)

Hamiltonian in the frame rotating at $ \frac{\omega_p}{2}$

$ H = H_A + H_B + H_C $

$ H_A = \Delta_p^a a^\dagger a + i \sqrt{2 \kappa_{ex1}} (a_{in} a^\dagger e^{-i \Delta_{in}t} - a_{in}^\dagger a e^{i \Delta_{in}t}) $

$ H_B = \Delta_p^b b^\dagger b + \frac{\Omega_p}{2} (e^{-i\theta_p} b^{\dagger 2} + e^{i\theta_p} b^2 )   $

$ H_J = J_0 (a^\dagger b + a b^\dagger)  $

when $ \Delta_p^a = \omega_a - \frac{\omega_p}{2} $ , $ \Delta_p^b = \omega_b - \frac{\omega_p}{2} , \Delta_{in} = \omega_{in} - \frac{\omega_p}{2} $

Pump field $ \Omega_p $ is created by driving a Second Harmonic Mode of Ring B with an external field, which means coupling between Second Harmonic Mode and external field.  

Pump field is only effective in phase - matching condition, which is $ \omega_p - 2 \omega_b = 0  $. Therefore, $ \Omega_p $ is only effective in Forward Signal and negligible in Backword Signal. Phase - matching condition is key of isolator.

Hamiltonian (FW case)

$ \frac{\Omega_p}{2} (e^{-i\theta_p} b^{\dagger 2} + e^{i\theta_p} b^2 ) $ term means quantum sqeezing on Ring B.

Applying Bogoliubov squeezing transformation $ b_s = \cosh(r_p) b + e^{-i\theta_p} \sinh(r_p) b^\dagger $ with the squeezing parameter $r_p = \frac{1}{4} \ln(\frac{1+\frac{\Omega_p}{\Delta_p^b}}{1-\frac{\Omega_p}{\Delta_p^b}}) $,

We can transform the Hamiltonian $H_{fw}$ to the squeezing picture and the Hamiltonian in the frame rotating at freuquency of $\Delta_{in}$ becomes

$ H_{fw} = \Delta_a a^\dagger a + i \sqrt{2 \kappa_{ex1}} (a_{in} a^\dagger - a_{in}^\dagger a ) + \Delta_b^s b_s^\dagger b_s + J_s (a^\dagger b_s + a b_s^\dagger)  $

where $ \Delta_a = \omega_a - \omega_{in}$, $ \Delta_b^s = \Delta_p^{bs} - \omega_{in}$, $ \Delta_p^{bs} = \Delta_p^b \sqrt{1-(\frac{\Omega_p}{\Delta_p^b})^2} $

In [ ]:
# Parameter normalized by